In [2]:
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns 
import numpy as np 
import scipy.stats 
import plotly.express as px
import sys
import os
import dotenv

In [111]:
current_dir = os.getcwd()
project_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
src_path = os.path.join(project_dir, 'src')
sys.path.append(src_path)

from utils import load_env_vars
load_env_vars()

# Importing the Dataset

In [69]:
us_immigration_csv = os.getenv('us_immigration_csv')
df = pd.read_csv(us_immigration_csv)

# Initial Observations & Cleaning

In [70]:
# First look
df.head()

,Year,Immigrants Obtaining Lawful Permanent Resident Status,Refugee Arrivals,Noncitizen Apprehensions,Noncitizen Removals,Noncitizen Returns
0,1980,"524,295","207,116","910,361","18,013","719,211"
1,1981,"595,014","159,252","975,780","17,379","823,875"
2,1982,"533,624","98,096","970,246","15,216","812,572"
3,1983,"550,052","61,218","1,251,357","19,211","931,600"
4,1984,"541,811","70,393","1,246,981","18,696","909,833"


In [173]:
# The columns needed to be renamed to a better format
df.rename(columns={'Immigrants Obtaining Lawful Permanent Resident Status': 'lawful_permanent_resident_obt',
                   'Refugee Arrivals': 'refugee_arrivals',
                   'Noncitizen Apprehensions': 'noncitizen_apprehensions',
                   'Noncitizen Removals': 'noncitizen_removals',
                   'Noncitizen Returns': 'noncitizen_returns',
                   'Year': 'year'}, inplace=True)

Let's check to see what the datatypes of the columns are

In [174]:
df.dtypes

year                             int64
lawful_permanent_resident_obt    int64
refugee_arrivals                 int64
noncitizen_apprehensions         int64
noncitizen_removals              int64
noncitizen_returns               int64
dtype: object

Most of the columns needed to be converted to integers

In [184]:
df.replace(',', '',regex=True, inplace=True)
df['lawful_permanent_resident_obt'] = df['lawful_permanent_resident_obt'].astype('int')
df['refugee_arrivals'] = df['refugee_arrivals'].astype('int')
df['noncitizen_apprehensions'] = df['noncitizen_apprehensions'].astype('int')
df['noncitizen_removals'] = df['noncitizen_removals'].astype('int')
df['noncitizen_returns'] = df['noncitizen_returns'].astype('int')
df.head()

,year,lawful_permanent_resident_obt,refugee_arrivals,noncitizen_apprehensions,noncitizen_removals,noncitizen_returns
0,1980,524295,207116,910361,18013,719211
1,1981,595014,159252,975780,17379,823875
2,1982,533624,98096,970246,15216,812572
3,1983,550052,61218,1251357,19211,931600
4,1984,541811,70393,1246981,18696,909833


Let's check if there are null values

In [185]:
df.isnull().sum()

year                             0
lawful_permanent_resident_obt    0
refugee_arrivals                 0
noncitizen_apprehensions         0
noncitizen_removals              0
noncitizen_returns               0
dtype: int64

# Presidents Dataset Cleaning

An interesting piece of information will be to see how the metrics differ among presidents and parties. This information is not currently in the dataset, so I'll take a dataset with that information and join it with the current dataset

In [200]:
presidents_csv = os.getenv('presidents_csv')
df_presidents = pd.read_csv(presidents_csv)

In [201]:
print('Rows and columns:', df_presidents.shape)
df_presidents

Rows and columns: (429, 6)


,year,name,party,term,salary,position_title
0,1789,"Washington,George",Unaffiliated,First,25000,PRESIDENT OF THE UNITED STATES
1,1790,"Washington,George",Unaffiliated,First,25000,PRESIDENT OF THE UNITED STATES
2,1791,"Washington,George",Unaffiliated,First,25000,PRESIDENT OF THE UNITED STATES
3,1792,"Washington,George",Unaffiliated,First,25000,PRESIDENT OF THE UNITED STATES
4,1793,"Washington,George",Unaffiliated,Second,25000,PRESIDENT OF THE UNITED STATES
...,...,...,...,...,...,...
424,2016,"Biden,Joseph Robinette,Jr.",Democratic,Second,230700,VICE PRESIDENT OF THE UNITED STATES
425,2017,"Pence,Michael Richard",Republican,First,230700,VICE PRESIDENT OF THE UNITED STATES
426,2018,"Pence,Michael Richard",Republican,First,230700,VICE PRESIDENT OF THE UNITED STATES
427,2019,"Pence,Michael Richard",Republican,First,235100,VICE PRESIDENT OF THE UNITED STATES


There are a series of filters that need to be put in:
1. The dataset contains data on vice presidents, which won't be needed. Those rows will be removed
2. Since the years in the immigration dataset are 1980-2021, the rows with other years in this dataset will be removed
3. The salary and position title columns won't be useful, so they'll be removed

In [202]:
df_presidents = df_presidents[df_presidents['position_title'] == 'PRESIDENT OF THE UNITED STATES']
df_presidents = df_presidents[df_presidents['year'] >= 1980]
df_presidents = df_presidents.loc[:, :'term']

In [203]:
df_presidents

,year,name,party,term
191,1980,"Carter,Jimmy Earl,Jr.",Democratic,First
192,1981,"Reagan,Ronald Wilson",Republican,First
193,1982,"Reagan,Ronald Wilson",Republican,First
194,1983,"Reagan,Ronald Wilson",Republican,First
195,1984,"Reagan,Ronald Wilson",Republican,First
196,1985,"Reagan,Ronald Wilson",Republican,Second
197,1986,"Reagan,Ronald Wilson",Republican,Second
198,1987,"Reagan,Ronald Wilson",Republican,Second
199,1988,"Reagan,Ronald Wilson",Republican,Second
200,1989,"Bush,George Herbert Walker",Republican,First


Finally, for aesthetics purposes, I'll replace the presidents' full names with the names they're known by

In [204]:
df_presidents['name'] = df_presidents['name'].replace('Carter,Jimmy Earl,Jr.', 'Jimmy Carter')
df_presidents['name'] = df_presidents['name'].replace('Reagan,Ronald Wilson', 'Ronald Reagan')
df_presidents['name'] = df_presidents['name'].replace('Bush,George Herbert Walker', 'George H.W. Bush')
df_presidents['name'] = df_presidents['name'].replace('Clinton,William Jefferson', 'Bill Clinton')
df_presidents['name'] = df_presidents['name'].replace('Bush,George Walker', 'George W. Bush')
df_presidents['name'] = df_presidents['name'].replace('Obama,Barack Hussein,II', 'Barack Obama')
df_presidents['name'] = df_presidents['name'].replace('Trump,Donald John', 'Donald Trump')

In [205]:
df_presidents.head()

,year,name,party,term
191,1980,Jimmy Carter,Democratic,First
192,1981,Ronald Reagan,Republican,First
193,1982,Ronald Reagan,Republican,First
194,1983,Ronald Reagan,Republican,First
195,1984,Ronald Reagan,Republican,First


# Joining Presidents Dataset With Immigration Dataset

Now that the presidents dataset is cleaned, it can be joined with the immigration dataset

In [213]:
df_final = df.merge(df_presidents, on='year')

In [214]:
# Rename 'name' column to 'president' to be more clear
df_final.rename(columns={'name': 'president'}, inplace=True)

In [215]:
df_final.head()

,year,lawful_permanent_resident_obt,refugee_arrivals,noncitizen_apprehensions,noncitizen_removals,noncitizen_returns,president,party,term
0,1980,524295,207116,910361,18013,719211,Jimmy Carter,Democratic,First
1,1981,595014,159252,975780,17379,823875,Ronald Reagan,Republican,First
2,1982,533624,98096,970246,15216,812572,Ronald Reagan,Republican,First
3,1983,550052,61218,1251357,19211,931600,Ronald Reagan,Republican,First
4,1984,541811,70393,1246981,18696,909833,Ronald Reagan,Republican,First


# EDA

In [87]:
from plotly.subplots import make_subplots

In [76]:
df.describe()

,Year,lawful_permanent_resident_obt,refugee_arrivals,noncitizen_apprehensions,noncitizen_removals,noncitizen_returns
count,42.000000,4.200000e+01,42.000000,4.200000e+01,42.000000,4.200000e+01
mean,2000.500000,9.142861e+05,73705.333333,1.136983e+06,179951.142857,8.464116e+05
std,12.267844,2.797462e+05,37346.453158,3.458046e+05,145240.084267,4.834266e+05
min,1980.000000,5.242950e+05,11454.000000,5.965600e+05,15216.000000,1.004540e+05
25%,1990.250000,6.657900e+05,54399.500000,8.944992e+05,33498.500000,3.594840e+05
50%,2000.500000,9.656640e+05,69914.500000,1.132329e+06,178963.500000,9.216950e+05
75%,2010.750000,1.061369e+06,85211.000000,1.318212e+06,323072.750000,1.103602e+06
max,2021.000000,1.826595e+06,207116.000000,1.865379e+06,432201.000000,1.675876e+06


In [91]:
lawful_permanent_resident_fig = px.line(df, x='Year', y='lawful_permanent_resident_obt', title='US Lawful Permanent Resident Obtainees By Year')
lawful_permanent_resident_fig.show()

In [85]:
refugee_arrivals_fig = px.line(df, x='Year', y='refugee_arrivals', title='US Refugee Arrivals By Year')
refugee_arrivals_fig.show()

In [93]:
noncitizen_apprehensions_fig = px.line(df, x='Year', y='noncitizen_apprehensions', title='US Non Citizen Apprehensions By Year')
noncitizen_apprehensions_fig.show()

In [86]:
noncitizen_removals_fig = px.line(df, x='Year', y='noncitizen_removals', title='US Noncitizen Removals By Year')
noncitizen_removals_fig.show()

In [94]:
noncitizen_returns_fig = px.line(df, x='Year', y='noncitizen_returns', title='US Noncitizen Returns By Year')
noncitizen_returns_fig.show()